## Step 1: Locate all DNA methylation beta value files

In this step, we identify all DNA methylation beta value files downloaded
from the GDC portal for both LUAD and LUSC cohorts.

We:
- Scan both TCGA-LUAD and TCGA-LUSC methylation directories
- Recursively search for files ending with:
  `methylation_array.sesame.level3betas.txt`
- Treat each file as one DNA methylation sample

Output:
- A list of methylation file paths for LUAD
- A list of methylation file paths for LUSC

In [1]:
from pathlib import Path

# Base directories
LUAD_METH_ROOT = Path(
    "/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUAD"
)

LUSC_METH_ROOT = Path(
    "/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUSC"
)

FILE_PATTERN = "**/*.methylation_array.sesame.level3betas.txt"

# Locate methylation files
luad_meth_files = sorted(LUAD_METH_ROOT.glob(FILE_PATTERN))
lusc_meth_files = sorted(LUSC_METH_ROOT.glob(FILE_PATTERN))

print(f"LUAD methylation files found: {len(luad_meth_files)}")
print(f"LUSC methylation files found: {len(lusc_meth_files)}")

# Sanity check: print one example from each
print("\nExample LUAD methylation file:")
print(luad_meth_files[0] if luad_meth_files else "None found")

print("\nExample LUSC methylation file:")
print(lusc_meth_files[0] if lusc_meth_files else "None found")

LUAD methylation files found: 473
LUSC methylation files found: 370

Example LUAD methylation file:
/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUAD/DNA_Methylation/Methylation_Beta_Value/02c085f9-c641-427c-9cb6-51720e3f4a3e/639097ee-9965-42ba-bbdb-d385d2f91fbe.methylation_array.sesame.level3betas.txt

Example LUSC methylation file:
/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUSC/DNA_Methylation/Methylation_Beta_Value/00118460-0c9d-4bf4-888a-3b76d2c067f5/66bf3ace-b7fe-4b00-ac72-eecefcbe1093.methylation_array.sesame.level3betas.txt


## Step 2: Decide the sample ID for DNA methylation (alignment key)

To ensure correct multimodal alignment, DNA methylation samples must use
the same identifier type as RNA.

For RNA, each sample is indexed using the GDC UUID corresponding to the
sample-level directory.

We apply the same rule to DNA methylation:
- Each methylation file is stored inside a UUID-named directory
- That directory name will be used as the `sample_id`

This guarantees:
- One-to-one correspondence between RNA and methylation samples
- Direct compatibility with `rna_train_ids.npy` and `rna_test_ids.npy`

Output:
- `sample_id` for every methylation file

In [2]:
from pathlib import Path

def extract_sample_id_from_path(filepath: Path):
    """
    Extract sample UUID from a GDC methylation file path.
    Assumes structure:
    .../<sample_uuid>/<file_uuid>.methylation_array.sesame.level3betas.txt
    """
    return filepath.parent.name


# Extract sample IDs
luad_meth_ids = [extract_sample_id_from_path(p) for p in luad_meth_files]
lusc_meth_ids = [extract_sample_id_from_path(p) for p in lusc_meth_files]

print(f"LUAD methylation sample IDs: {len(luad_meth_ids)}")
print(f"LUSC methylation sample IDs: {len(lusc_meth_ids)}")

# Show examples
print("\nExample LUAD methylation IDs:")
for sid in luad_meth_ids[:5]:
    print(sid)

print("\nExample LUSC methylation IDs:")
for sid in lusc_meth_ids[:5]:
    print(sid)

print("\nDuplicate check:")
print("LUAD duplicates:", len(luad_meth_ids) - len(set(luad_meth_ids)))
print("LUSC duplicates:", len(lusc_meth_ids) - len(set(lusc_meth_ids)))

LUAD methylation sample IDs: 473
LUSC methylation sample IDs: 370

Example LUAD methylation IDs:
02c085f9-c641-427c-9cb6-51720e3f4a3e
02eee71c-fe4a-40b9-8e5f-a8691e8013e6
034ddf29-0552-44b9-97b4-b9a9a38b6ca0
03639951-3a42-49d5-ae8f-e26c128223c7
03a635d5-bd6b-489b-ad60-b2d920af9fe6

Example LUSC methylation IDs:
00118460-0c9d-4bf4-888a-3b76d2c067f5
005507e0-a9f1-4d88-8d87-d10d5fdd5e69
01232c9e-b486-4aa5-ae14-70be61869455
015da321-d6f6-40ab-aeb6-6bccd036d318
02dc0520-f646-486c-888c-364b83cb95ba

Duplicate check:
LUAD duplicates: 0
LUSC duplicates: 0


## Step 3: Parse a single DNA methylation beta value file

In this step, we inspect ONE DNA methylation file to confirm:
- Rows correspond to CpG probe IDs (e.g., cg00000029)
- The file contains beta values in the range [0, 1]
- The format is consistent across samples
- The pattern of missing values

This step is for validation only.
No matrix construction is performed yet.

Output:
- Confirmed methylation file structure

In [3]:
import pandas as pd

# Pick one example file (LUAD)
sample_file = luad_meth_files[0]
sample_id = sample_file.parent.name

print("Sample ID:", sample_id)
print("File:", sample_file)

# Read file
df = pd.read_csv(sample_file, sep="\t")

print("\nParsed shape:", df.shape)
print("Columns:", list(df.columns))

print("\nFirst 10 rows:")
display(df.head(10))

# Basic checks
print("\nMissing values per column:")
print(df.isna().sum())

# If CpG IDs are in the first column, check format
first_col = df.columns[0]
print("\nFirst column name:", first_col)
print("First 5 CpG IDs:", df[first_col].head().tolist())

# Beta value sanity check (if single beta column)
numeric_cols = df.select_dtypes(include=["float", "int"]).columns
print("\nNumeric columns:", list(numeric_cols))

if len(numeric_cols) > 0:
    print("\nBeta value range:")
    print("min =", df[numeric_cols[0]].min(),
          "max =", df[numeric_cols[0]].max())

Sample ID: 02c085f9-c641-427c-9cb6-51720e3f4a3e
File: /home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUAD/DNA_Methylation/Methylation_Beta_Value/02c085f9-c641-427c-9cb6-51720e3f4a3e/639097ee-9965-42ba-bbdb-d385d2f91fbe.methylation_array.sesame.level3betas.txt

Parsed shape: (486426, 2)
Columns: ['cg00000029', '0.0921292799905492']

First 10 rows:


,cg00000029,0.0921292799905492
0,cg00000108,0.967265
1,cg00000109,0.928745
2,cg00000165,0.388836
3,cg00000236,0.923432
4,cg00000289,0.750244
5,cg00000292,0.877350
6,cg00000321,0.224977
7,cg00000363,0.133819
8,cg00000622,0.014318
9,cg00000658,0.869611



Missing values per column:
cg00000029                0
0.0921292799905492    71747
dtype: int64

First column name: cg00000029
First 5 CpG IDs: ['cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289']

Numeric columns: ['0.0921292799905492']

Beta value range:
min = 0.0065479824593137 max = 0.993317175367617


## Step 4: Extract beta values for a single DNA methylation sample

Based on inspection, each methylation file contains:
- Column 0: CpG probe ID
- Column 1: beta value in [0, 1]

In this step, we:
- Read the file without assuming a header
- Explicitly name columns
- Extract a clean `{CpG → beta}` vector for one sample

This extraction logic will be reused for all samples.

Output:
- A validated beta vector for one methylation sample

In [4]:
import pandas as pd
import numpy as np

# Use the same sample file you inspected
sample_file = luad_meth_files[0]
sample_id = sample_file.parent.name

print("Sample ID:", sample_id)
print("File:", sample_file)

# Read WITHOUT header and assign column names
df = pd.read_csv(
    sample_file,
    sep="\t",
    header=None,
    names=["cpg_id", "beta"]
)

print("\nParsed shape:", df.shape)
display(df.head(10))

# Basic validation
print("\nBeta stats:")
print("Missing betas:", df["beta"].isna().sum())
print("Min beta:", df["beta"].min())
print("Max beta:", df["beta"].max())

# Ensure CpG IDs look correct
print("\nFirst 5 CpG IDs:", df["cpg_id"].head().tolist())

# Create beta vector indexed by CpG
beta_series = df.set_index("cpg_id")["beta"]

print("\nBeta vector length:", beta_series.shape[0])

Sample ID: 02c085f9-c641-427c-9cb6-51720e3f4a3e
File: /home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/01_download/methylation/TCGA-LUAD/DNA_Methylation/Methylation_Beta_Value/02c085f9-c641-427c-9cb6-51720e3f4a3e/639097ee-9965-42ba-bbdb-d385d2f91fbe.methylation_array.sesame.level3betas.txt

Parsed shape: (486427, 2)


,cpg_id,beta
0,cg00000029,0.092129
1,cg00000108,0.967265
2,cg00000109,0.928745
3,cg00000165,0.388836
4,cg00000236,0.923432
5,cg00000289,0.750244
6,cg00000292,0.877350
7,cg00000321,0.224977
8,cg00000363,0.133819
9,cg00000622,0.014318



Beta stats:
Missing betas: 71747
Min beta: 0.0065479824593137
Max beta: 0.993317175367617

First 5 CpG IDs: ['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236']

Beta vector length: 486427


## Step 5: Build raw DNA methylation matrix (all samples)

Using the validated beta extraction logic, we construct the raw DNA methylation
matrix across all samples.

Procedure:
- Extract CpG → beta vectors for each sample
- Take the union of all CpG probes
- Build a samples × CpGs matrix
- Missing CpGs are represented as NaN

Output:
- `X_meth_raw`
- `meth_sample_ids`
- `meth_cpg_ids`

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

OUT_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Combine LUAD + LUSC files and IDs
meth_files = luad_meth_files + lusc_meth_files
meth_ids = [p.parent.name for p in meth_files]

print("Total methylation samples:", len(meth_files))

# ---- Step 1: extract beta vectors ----
beta_dict = {}

for file_path, sample_id in tqdm(zip(meth_files, meth_ids), total=len(meth_files)):
    df = pd.read_csv(
        file_path,
        sep="\t",
        header=None,
        names=["cpg_id", "beta"]
    )
    beta_series = df.set_index("cpg_id")["beta"]
    beta_dict[sample_id] = beta_series

# ---- Step 2: build DataFrame (union of CpGs) ----
meth_df = pd.DataFrame.from_dict(beta_dict, orient="index")

print("\nRaw methylation matrix shape (samples × CpGs):", meth_df.shape)

# ---- Step 3: save raw outputs ----
X_meth_raw = meth_df.values.astype(np.float32)
meth_sample_ids = meth_df.index.to_numpy()
meth_cpg_ids = meth_df.columns.to_numpy()

np.save(OUT_DIR / "meth_raw.npy", X_meth_raw)
np.save(OUT_DIR / "meth_sample_ids.npy", meth_sample_ids)
np.save(OUT_DIR / "meth_cpg_ids.npy", meth_cpg_ids)

print("\nSaved:")
print(" - meth_raw.npy")
print(" - meth_sample_ids.npy")
print(" - meth_cpg_ids.npy")

# ---- Basic QC ----
print("\nQC checks:")
print("Any duplicate samples:", len(meth_sample_ids) != len(set(meth_sample_ids)))
print("Total missing values:", np.isnan(X_meth_raw).sum())

Total methylation samples: 843


100%|██████████| 843/843 [03:25<00:00,  4.10it/s]



Raw methylation matrix shape (samples × CpGs): (843, 486427)

Saved:
 - meth_raw.npy
 - meth_sample_ids.npy
 - meth_cpg_ids.npy

QC checks:
Any duplicate samples: False
Total missing values: 60695512


## Step 6 : Map file UUIDs to TCGA patient IDs (cross-modality alignment)

The UUID folder names in GDC downloads are file-level identifiers and do not match
across modalities (RNA vs methylation). Therefore, direct intersection is zero.

To align modalities correctly, we map:
- file_uuid → TCGA case submitter ID (e.g., TCGA-XX-YYYY-....)
Then define:
- patient_id = first 12 characters (TCGA-XX-YYYY)

Outputs:
- mapping table: file_uuid → case_submitter_id → patient_id
- RNA train/test patient IDs
- methylation train/test matrices aligned to RNA split via patient_id


In [7]:
import numpy as np
import pandas as pd
import requests
from pathlib import Path
from tqdm import tqdm

RNA_DIR  = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/rna")
METH_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")

# Load the file-level IDs you currently have
rna_train_file_ids = np.load(RNA_DIR / "rna_train_ids.npy", allow_pickle=True).astype(str)
rna_test_file_ids  = np.load(RNA_DIR / "rna_test_ids.npy", allow_pickle=True).astype(str)

meth_file_ids = np.load(METH_DIR / "meth_sample_ids.npy", allow_pickle=True).astype(str)

all_file_ids = np.unique(np.concatenate([rna_train_file_ids, rna_test_file_ids, meth_file_ids]))
print("Total unique file UUIDs to map:", len(all_file_ids))

GDC_FILES_ENDPOINT = "https://api.gdc.cancer.gov/files"

def chunk_list(x, chunk_size=2000):
    for i in range(0, len(x), chunk_size):
        yield x[i:i+chunk_size]

def fetch_gdc_mapping(file_id_chunk):
    # Request file_id -> cases.submitter_id
    payload = {
        "filters": {
            "op": "in",
            "content": {"field": "file_id", "value": list(file_id_chunk)}
        },
        "fields": "file_id,cases.submitter_id",
        "format": "JSON",
        "size": len(file_id_chunk)
    }
    r = requests.post(GDC_FILES_ENDPOINT, json=payload, timeout=120)
    r.raise_for_status()
    hits = r.json()["data"]["hits"]
    rows = []
    for h in hits:
        file_id = h.get("file_id")
        cases = h.get("cases", [])
        case_submitter_id = cases[0].get("submitter_id") if len(cases) > 0 else None
        rows.append((file_id, case_submitter_id))
    return rows

# ---- Build mapping table ----
mapping_rows = []
for ch in tqdm(list(chunk_list(all_file_ids, 2000))):
    mapping_rows.extend(fetch_gdc_mapping(ch))

map_df = pd.DataFrame(mapping_rows, columns=["file_id", "case_submitter_id"])
map_df["patient_id"] = map_df["case_submitter_id"].astype(str).str.slice(0, 12)

print("\nMapping head:")
display(map_df.head())

print("\nMapped file_ids:", map_df["case_submitter_id"].notna().sum(), "/", len(map_df))

# Save mapping
OUT_MAP = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/id_maps")
OUT_MAP.mkdir(parents=True, exist_ok=True)
map_df.to_csv(OUT_MAP / "file_uuid_to_patient_id.csv", index=False)
print("\nSaved mapping to:", OUT_MAP / "file_uuid_to_patient_id.csv")

# ---- Translate RNA split from file_uuid -> patient_id ----
file_to_patient = dict(zip(map_df["file_id"], map_df["patient_id"]))

rna_train_patient = np.array([file_to_patient.get(fid) for fid in rna_train_file_ids], dtype=object)
rna_test_patient  = np.array([file_to_patient.get(fid) for fid in rna_test_file_ids], dtype=object)

# Drop any unmapped (should be very few, but safe)
rna_train_patient = rna_train_patient[pd.notna(rna_train_patient)]
rna_test_patient  = rna_test_patient[pd.notna(rna_test_patient)]

print("\nRNA split converted to patient IDs:")
print("Train patients:", len(rna_train_patient), "unique:", len(np.unique(rna_train_patient)))
print("Test patients :", len(rna_test_patient),  "unique:", len(np.unique(rna_test_patient)))

np.save(RNA_DIR / "rna_train_patient_ids.npy", rna_train_patient)
np.save(RNA_DIR / "rna_test_patient_ids.npy", rna_test_patient)

print("Saved:")
print(" - rna_train_patient_ids.npy")
print(" - rna_test_patient_ids.npy")

# ---- Now align methylation using patient_id ----
X_meth_raw = np.load(METH_DIR / "meth_raw.npy")
meth_file_ids = np.load(METH_DIR / "meth_sample_ids.npy", allow_pickle=True).astype(str)

meth_patient = np.array([file_to_patient.get(fid) for fid in meth_file_ids], dtype=object)

# Keep only mapped methylation rows
mapped_mask = pd.notna(meth_patient)
X_meth_raw_mapped = X_meth_raw[mapped_mask]
meth_patient_mapped = meth_patient[mapped_mask]

print("\nMethylation mapped rows:", X_meth_raw_mapped.shape[0], "/", X_meth_raw.shape[0])

# If multiple methylation files per patient exist, keep the first occurrence (simple, reproducible)
seen = {}
keep_idx = []
for i, pid in enumerate(meth_patient_mapped):
    if pid not in seen:
        seen[pid] = i
        keep_idx.append(i)

keep_idx = np.array(keep_idx, dtype=int)
X_meth_unique = X_meth_raw_mapped[keep_idx]
meth_patient_unique = meth_patient_mapped[keep_idx]

print("Methylation unique patients:", len(meth_patient_unique))

# Build train/test sets using RNA patient split
train_set = set(rna_train_patient.tolist())
test_set  = set(rna_test_patient.tolist())

train_idx = np.array([i for i, pid in enumerate(meth_patient_unique) if pid in train_set], dtype=int)
test_idx  = np.array([i for i, pid in enumerate(meth_patient_unique) if pid in test_set], dtype=int)

X_meth_train_raw = X_meth_unique[train_idx]
X_meth_test_raw  = X_meth_unique[test_idx]

meth_train_patient_ids = meth_patient_unique[train_idx]
meth_test_patient_ids  = meth_patient_unique[test_idx]

print("\nAligned methylation (by patient_id):")
print("Train:", X_meth_train_raw.shape)
print("Test :", X_meth_test_raw.shape)

np.save(METH_DIR / "meth_train_raw.npy", X_meth_train_raw)
np.save(METH_DIR / "meth_test_raw.npy", X_meth_test_raw)
np.save(METH_DIR / "meth_train_patient_ids.npy", meth_train_patient_ids)
np.save(METH_DIR / "meth_test_patient_ids.npy", meth_test_patient_ids)

print("\nSaved corrected aligned methylation:")
print(" - meth_train_raw.npy")
print(" - meth_test_raw.npy")
print(" - meth_train_patient_ids.npy")
print(" - meth_test_patient_ids.npy")

Total unique file UUIDs to map: 1668


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


Mapping head:


,file_id,case_submitter_id,patient_id
0,ada4ae4b-1959-49a1-b3db-59675ba2ade0,TCGA-96-A4JK,TCGA-96-A4JK
1,343f1771-4a3d-4a03-9eb0-319dada0b373,TCGA-85-A4QQ,TCGA-85-A4QQ
2,0bde4f32-b0ce-472f-851a-b4571916b502,TCGA-85-A4QQ,TCGA-85-A4QQ
3,c6eae9c3-625e-43c5-a969-d408ea922d55,TCGA-O2-A5IB,TCGA-O2-A5IB
4,17ddff0c-a5c2-4f82-b638-fe13c20de1d3,TCGA-O2-A5IB,TCGA-O2-A5IB



Mapped file_ids: 1668 / 1668

Saved mapping to: /home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/id_maps/file_uuid_to_patient_id.csv

RNA split converted to patient IDs:
Train patients: 660 unique: 660
Test patients : 165 unique: 165
Saved:
 - rna_train_patient_ids.npy
 - rna_test_patient_ids.npy

Methylation mapped rows: 843 / 843
Methylation unique patients: 828

Aligned methylation (by patient_id):
Train: (660, 486427)
Test : (165, 486427)

Saved corrected aligned methylation:
 - meth_train_raw.npy
 - meth_test_raw.npy
 - meth_train_patient_ids.npy
 - meth_test_patient_ids.npy


## Step 7: Handle missing values (required for DNA methylation)

DNA methylation beta matrices typically contain missing CpG probes (NaNs) due to
array QC and probe masking. Most ML models cannot handle NaNs, so we must:

1) Remove CpGs with high missingness (learned from TRAIN only)
   - Compute missing rate per CpG across training samples
   - Drop CpGs missing in > 20% of training samples (threshold can be 10–20%)

2) Impute remaining missing values (TRAIN only, then apply to TEST)
   - For each remaining CpG, compute the median beta value on TRAIN samples
   - Fill NaNs in TRAIN and TEST using the TRAIN CpG medians

This avoids data leakage and produces complete matrices for downstream PCA/ML.

Outputs:
- `meth_train_complete.npy`
- `meth_test_complete.npy`
- `meth_cpg_keep_mask.npy`
- `meth_cpg_ids_kept.npy`
- `meth_train_cpg_medians.npy`

In [8]:
import numpy as np
from pathlib import Path

METH_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")

# Load aligned raw methylation (already matched to RNA split by patient_id)
X_train = np.load(METH_DIR / "meth_train_raw.npy")  # shape: (660, CpGs)
X_test  = np.load(METH_DIR / "meth_test_raw.npy")   # shape: (165, CpGs)
cpg_ids  = np.load(METH_DIR / "meth_cpg_ids.npy", allow_pickle=True)

print("Raw aligned methylation:")
print("Train:", X_train.shape, " Test:", X_test.shape)

# -----------------------------
# 7A) Filter CpGs by missingness (TRAIN ONLY)
# -----------------------------
missing_rate_train = np.isnan(X_train).mean(axis=0)  # per CpG
THRESH = 0.20  # 20% missingness threshold (can switch to 0.10 for stricter filtering)

cpg_keep_mask = missing_rate_train <= THRESH
kept_cpg_ids = cpg_ids[cpg_keep_mask]

X_train_f = X_train[:, cpg_keep_mask]
X_test_f  = X_test[:,  cpg_keep_mask]

print("\nAfter missingness filtering (train-only):")
print("Kept CpGs:", kept_cpg_ids.shape[0], "/", cpg_ids.shape[0])
print("Train:", X_train_f.shape, " Test:", X_test_f.shape)

# -----------------------------
# 7B) Impute remaining NaNs (TRAIN medians, apply to TRAIN + TEST)
# -----------------------------
# Compute CpG-wise medians on TRAIN (ignoring NaNs)
train_cpg_medians = np.nanmedian(X_train_f, axis=0)

# If any CpG is entirely NaN in train (rare after filtering), nanmedian returns NaN
# We drop those CpGs to be safe.
all_nan_mask = np.isnan(train_cpg_medians)
if all_nan_mask.any():
    print("\nWarning: CpGs with all-NaN medians found:", int(all_nan_mask.sum()))
    keep2 = ~all_nan_mask
    train_cpg_medians = train_cpg_medians[keep2]
    kept_cpg_ids = kept_cpg_ids[keep2]
    X_train_f = X_train_f[:, keep2]
    X_test_f  = X_test_f[:,  keep2]
    print("After dropping all-NaN CpGs:")
    print("Kept CpGs:", kept_cpg_ids.shape[0])

# Impute TRAIN
X_train_complete = X_train_f.copy()
train_nan = np.isnan(X_train_complete)
if train_nan.any():
    X_train_complete[train_nan] = np.take(train_cpg_medians, np.where(train_nan)[1])

# Impute TEST using TRAIN medians
X_test_complete = X_test_f.copy()
test_nan = np.isnan(X_test_complete)
if test_nan.any():
    X_test_complete[test_nan] = np.take(train_cpg_medians, np.where(test_nan)[1])

# Final checks
print("\nFinal completeness check:")
print("Train NaNs:", int(np.isnan(X_train_complete).sum()))
print("Test  NaNs:", int(np.isnan(X_test_complete).sum()))

# Save outputs
np.save(METH_DIR / "meth_train_complete.npy", X_train_complete.astype(np.float32))
np.save(METH_DIR / "meth_test_complete.npy", X_test_complete.astype(np.float32))
np.save(METH_DIR / "meth_cpg_keep_mask.npy", cpg_keep_mask)
np.save(METH_DIR / "meth_cpg_ids_kept.npy", kept_cpg_ids)
np.save(METH_DIR / "meth_train_cpg_medians.npy", train_cpg_medians.astype(np.float32))

print("\nSaved:")
print(" - meth_train_complete.npy")
print(" - meth_test_complete.npy")
print(" - meth_cpg_keep_mask.npy")
print(" - meth_cpg_ids_kept.npy")
print(" - meth_train_cpg_medians.npy")

Raw aligned methylation:
Train: (660, 486427)  Test: (165, 486427)

After missingness filtering (train-only):
Kept CpGs: 411294 / 486427
Train: (660, 411294)  Test: (165, 411294)

Final completeness check:
Train NaNs: 0
Test  NaNs: 0

Saved:
 - meth_train_complete.npy
 - meth_test_complete.npy
 - meth_cpg_keep_mask.npy
 - meth_cpg_ids_kept.npy
 - meth_train_cpg_medians.npy


## Step 8: Transform beta values to M-values (recommended)

Beta values are bounded in [0, 1], which can cause skewness.
A common transformation is the M-value:

$$
M = \log_2 \left( \frac{\beta + \epsilon}{1 - \beta + \epsilon} \right)
$$

This produces values on an unbounded scale that often behaves more normally,
which improves PCA and many ML models.

Outputs:
- `meth_train_mvals.npy`
- `meth_test_mvals.npy`

In [9]:
import numpy as np
from pathlib import Path

METH_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")

X_train_beta = np.load(METH_DIR / "meth_train_complete.npy")  # (660, CpGs)
X_test_beta  = np.load(METH_DIR / "meth_test_complete.npy")   # (165, CpGs)

print("Beta matrices loaded:")
print("Train:", X_train_beta.shape, " Test:", X_test_beta.shape)

eps = 1e-6  # small constant to prevent log(0)

# Clip betas slightly away from 0 and 1 for numerical stability
X_train_b = np.clip(X_train_beta, eps, 1 - eps)
X_test_b  = np.clip(X_test_beta,  eps, 1 - eps)

# M-value transform
X_train_m = np.log2(X_train_b / (1.0 - X_train_b)).astype(np.float32)
X_test_m  = np.log2(X_test_b  / (1.0 - X_test_b)).astype(np.float32)

print("\nM-value stats check:")
print("Train M min/max:", float(X_train_m.min()), float(X_train_m.max()))
print("Test  M min/max:", float(X_test_m.min()),  float(X_test_m.max()))

np.save(METH_DIR / "meth_train_mvals.npy", X_train_m)
np.save(METH_DIR / "meth_test_mvals.npy", X_test_m)

print("\nSaved:")
print(" - meth_train_mvals.npy")
print(" - meth_test_mvals.npy")

Beta matrices loaded:
Train: (660, 411294)  Test: (165, 411294)

M-value stats check:
Train M min/max: -8.065641403198242 8.509053230285645
Test  M min/max: -8.020096778869629 7.939126491546631

Saved:
 - meth_train_mvals.npy
 - meth_test_mvals.npy


## Step 9: Standardize methylation features (train-only)

After M-value transformation, CpGs still have different scales.
Most ML models and PCA require features to have comparable scale.

Procedure:
- Fit StandardScaler on TRAIN methylation M-values only
- Apply the same scaler to TEST

Output:
- meth_train_scaled.npy
- meth_test_scaled.npy
- meth_standard_scaler.joblib

In [10]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path

METH_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")

# Load M-values
X_train_m = np.load(METH_DIR / "meth_train_mvals.npy")  # (660, CpGs)
X_test_m  = np.load(METH_DIR / "meth_test_mvals.npy")   # (165, CpGs)

print("M-value shapes:")
print("Train:", X_train_m.shape, " Test:", X_test_m.shape)

# Fit scaler on TRAIN only
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train_m).astype(np.float32)
X_test_scaled  = scaler.transform(X_test_m).astype(np.float32)

# Sanity check
print("\nScaled data check:")
print("Train mean (first 5 CpGs):", X_train_scaled.mean(axis=0)[:5])
print("Train std  (first 5 CpGs):", X_train_scaled.std(axis=0)[:5])

# Save outputs
np.save(METH_DIR / "meth_train_scaled.npy", X_train_scaled)
np.save(METH_DIR / "meth_test_scaled.npy", X_test_scaled)
joblib.dump(scaler, METH_DIR / "meth_standard_scaler.joblib")

print("\nSaved:")
print(" - meth_train_scaled.npy")
print(" - meth_test_scaled.npy")
print(" - meth_standard_scaler.joblib")

M-value shapes:
Train: (660, 411294)  Test: (165, 411294)

Scaled data check:
Train mean (first 5 CpGs): [-2.6641469e-09  1.8423254e-08  5.8249996e-09  4.2084491e-08
 -4.6058133e-08]
Train std  (first 5 CpGs): [0.9999993  0.9999998  1.0000004  0.9999999  0.99999994]

Saved:
 - meth_train_scaled.npy
 - meth_test_scaled.npy
 - meth_standard_scaler.joblib


## Step 10: Dimensionality reduction with PCA (methylation)

DNA methylation contains hundreds of thousands of CpG probes,
many of which are highly correlated.

Procedure:
- Fit PCA on TRAIN methylation M-values only
- Select the smallest number of components explaining ≥ 90% variance
- Transform both train and test using the same PCA model

This improves:
- computational efficiency
- numerical stability
- generalization performance

Output:
- meth_train_pca.npy
- meth_test_pca.npy
- meth_pca_explained_variance.npy
- meth_pca_model.joblib

In [11]:
import numpy as np
from sklearn.decomposition import PCA
import joblib
from pathlib import Path

METH_DIR = Path("/home/steps4growth/gmriechi/Lung-Cancer-Subtyping/Data/02_preprocessed/methylation")

# Load scaled methylation
X_train = np.load(METH_DIR / "meth_train_scaled.npy")  # (660, CpGs)
X_test  = np.load(METH_DIR / "meth_test_scaled.npy")   # (165, CpGs)

print("Scaled methylation shapes:")
print("Train:", X_train.shape, " Test:", X_test.shape)

# Fit PCA on TRAIN only
pca = PCA(n_components=0.90, svd_solver="full", random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

# Diagnostics
n_components = pca.n_components_
cum_var = pca.explained_variance_ratio_.sum()

print(f"\nNumber of PCA components for 90% variance: {n_components}")
print(f"Cumulative variance explained: {cum_var:.4f}")

print("\nPCA-transformed shapes:")
print("X_meth_train_pca:", X_train_pca.shape)
print("X_meth_test_pca :", X_test_pca.shape)

# Save outputs
np.save(METH_DIR / "meth_train_pca.npy", X_train_pca.astype(np.float32))
np.save(METH_DIR / "meth_test_pca.npy", X_test_pca.astype(np.float32))
np.save(METH_DIR / "meth_pca_explained_variance.npy", pca.explained_variance_ratio_)
joblib.dump(pca, METH_DIR / "meth_pca_model.joblib")

print("\nSaved:")
print(" - meth_train_pca.npy")
print(" - meth_test_pca.npy")
print(" - meth_pca_explained_variance.npy")
print(" - meth_pca_model.joblib")

Scaled methylation shapes:
Train: (660, 411294)  Test: (165, 411294)

Number of PCA components for 90% variance: 402
Cumulative variance explained: 0.9001

PCA-transformed shapes:
X_meth_train_pca: (660, 402)
X_meth_test_pca : (165, 402)

Saved:
 - meth_train_pca.npy
 - meth_test_pca.npy
 - meth_pca_explained_variance.npy
 - meth_pca_model.joblib
